In [ ]:
!starcluster start mycluster
!starcluster sshmaster mycluster

In [1]:
from os.path import expanduser
from IPython.parallel import Client

url_file = expanduser('C:\Users\Balaji\.starcluster\ipcluster\SecurityGroup:@sc-mycluster-us-east-1.json')
sshkey = expanduser('C:\Anaconda\Galvanize\DataEngineering\.ssh\StarCluster.rsa')
client = Client(url_file, sshkey = sshkey)

C:\Anaconda\lib\site-packages\paramiko\client.py:580: UserWarning: Unknown ssh-rsa host key for ec2-54-173-50-18.compute-1.amazonaws.com: f1f9d64c00ec85a1c79ad29aaa2d53a9
  (key.get_name(), hostname, hexlify(key.get_fingerprint())))


In [2]:
dview = client.direct_view()
len(client.ids)

3

In [9]:
!starcluster put mycluster --user sgeadmin 20news-bydate.tar.gz /mnt/sgeadmin/

StarCluster - (http://star.mit.edu/cluster) (v. 0.95.6)
Software Tools for Academics and Researchers (STAR)
Please submit bug reports to starcluster@mit.edu

!!! ERROR - Local file or directory does not exist: 20news-bydate.tar.gz


In [3]:
all_engines = client[:]
all_engines

<DirectView [0, 1, 2]>

In [4]:
%%px

import os
import socket
print("This is running in process with pid {0} on host '{1}'.".format(
      os.getpid(), socket.gethostname()))

TypeError: expected string or buffer

In [49]:
def my_sum(a, b):
    return a + b

my_sum_apply_results = all_engines.apply(my_sum, 11, 31)
my_sum_apply_results
my_sum_apply_results.get()

TypeError: expected string or buffer

In [52]:
%px a = 1

TypeError: expected string or buffer

In [9]:
lv = client.load_balanced_view()

In [10]:
def slow_square(x):
    import time
    time.sleep(2)
    return x ** 2

In [11]:
result = lv.apply(slow_square, 4)

In [12]:
result

<AsyncResult: finished>

In [13]:
result.ready()

True

In [14]:
result.get()

16

In [15]:
results = lv.map(slow_square, [0, 1, 2, 3])
results

<AsyncMapResult: slow_square>

In [16]:
results.ready()

False

In [17]:
results.progress

1

In [18]:
for r in results:
    print(r)

0
1
4
9


In [19]:
%px import numpy as np

In [20]:
%%px --targets=-1  
# replace with one_engine_by_host.values()

# Cleanup any existing file from past session (necessary for windows)
import os
if os.path.exists('small.mmap'):
    os.unlink('small.mmap')

mm_w = np.memmap('small.mmap', shape=10, dtype=np.float32, mode='w+')
print(mm_w)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [21]:
%%px

mm_r = np.memmap('small.mmap', dtype=np.float32, mode='r+')
print(mm_r)

[stdout:0] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[stdout:1] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[stdout:2] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [22]:
%%px --targets=-1  
# replace with one_engine_by_host.values()

mm_w[0] = 42
print(mm_w)
print(mm_r)

[ 42.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
[ 42.   0.   0.   0.   0.   0.   0.   0.   0.   0.]


In [23]:
%px print(mm_r)

[stdout:0] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[stdout:1] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[stdout:2] [ 42.   0.   0.   0.   0.   0.   0.   0.   0.   0.]


In [25]:
%%px --targets=1

mm_r[1] = 43

In [26]:
%%px
print(mm_r)

[stdout:0] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[stdout:1] [  0.  43.   0.   0.   0.   0.   0.   0.   0.   0.]
[stdout:2] [ 42.   0.   0.   0.   0.   0.   0.   0.   0.   0.]


In [27]:
%%px
print("sum={0:.3}, mean={1:.3}, std={2:.3}".format(
    mm_r.sum(), np.mean(mm_r), np.std(mm_r)))

[stdout:0] sum=42., mean=4.2, std=12.6
[stdout:1] sum=43., mean=4.3, std=12.9
[stdout:2] sum=42., mean=4.2, std=12.6


In [37]:
%px

def get_host_free_memory(client):
    """Free memory on each host of the cluster in MB."""
    all_engines = client[:]
    def hostname():
        import socket
        return socket.gethostname()
    
    hostnames = all_engines.apply(hostname).get_dict()
    one_engine_per_host = dict((hostname, engine_id)
                               for engine_id, hostname
                               in hostnames.items())

    def host_free_memory():
        import psutil
        return psutil.virtual_memory().free / 1e6
    
    
    host_mem = client[one_engine_per_host.values()].apply(
        host_free_memory).get_dict()
    
    return dict((hostnames[eid], m) for eid, m in host_mem.items())

In [44]:
%%px --targets=1

!pip install psutil

KeyboardInterrupt: 

In [ ]:
get_host_free_memory(client)